In [26]:
%%html
<script>
    // AUTORUN ALL CELLS ON NOTEBOOK-LOAD! https://stackoverflow.com/questions/31984196/ipython-notebook-run-all-cells-on-open
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            $(jupyter.events).on("kernel_ready.Kernel", function () {
                console.log("Auto-running all cells-below...");
               //IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+2)
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
            });
           // $('#header-container').hide();
          //$('#maintoolbar').hide();   
        }
    );

</script>

In [27]:
from IPython.display import HTML, clear_output,Javascript, display
from ipywidgets import widgets
from ipywidgets import Button, GridBox, Layout, ButtonStyle, Box, VBox
import requests
import json
import pandas as pd
from pprintpp import pprint as pp
from pandas.io.json import json_normalize
import time
import datetime as dt
import pytz
#pd.set_option('display.max_rows', 999)
# I wish i knew how to always hide this cell !

HTML('''
<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>''')



# Welcome 🙋🏼‍♀️

### Purpose
API tool to do quick API calls to the Canvas instance using your Canvas API token 
If you are very interested in the Code that is behind this click the 'Show Code' Button that is at the top of this page.


#### API documentation references 
 - [CANVAS API DOC](https://canvas.instructure.com/doc/api/index.html)
 - [Requests Python Library](http://docs.python-requests.org/en/master/)

### ⚡️ Quick Directions ⚡️

1. enter in api token ( don't know what that is? please see: https://community.canvaslms.com/docs/DOC-10806 )
3. select canvas environment 
4. click submit ( make sure the top turns green )
5. scroll down to see the currently available reporting


In [28]:
error = widgets.Output()

API_widget = widgets.Text(placeholder='API TOKEN',
                       layout=Layout(width='90%',align_items='center', grid_area='main'))


Authentication = Button(
    value=False,
    description='NOT Authorized',
    disabled=True,
    button_style='warning', # 'success', 'info', 'warning', 'danger' or ''
    icon='unlock',
    layout=Layout(width='auto', grid_area='header')
)


Submit = Button(
    value=False,
    description='Submit',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    icon='',
    layout=Layout(width='auto', grid_area='footer')
)



# for the env // Domain
environ = widgets.Select(
    options=['Production', 'Test', 'Beta'],
    value='Production',
    description='Environment:',
    disabled=False,
    rows=3,
    layout=Layout(width='auto', grid_area='sidebar')
)

environs = {'Production':'https://canvas.upenn.edu/api/v1/', 'Test':'b', 'Beta':'c'}



def set_up_forms():
    display(Javascript('IPython.notebook.execute_cell_range(IPython.notebook.get_selected_index()+1, IPython.notebook.get_selected_index()+10)'))

def authorize():
    # check api key 
    global headers
    headers = {
        'Authorization': 'Bearer %s' % (API_widget.value) ,
    }
    global domain
    domain = environs[environ.value]
    
    x = test_api() # returns status code of test response
    
    if API_widget.value and x ==200:
    # if good:
        
        # UI changes
        Authentication.icon = 'lock'
        Authentication.description = 'Authorized'
        Authentication.button_style ='success'
        set_up_forms()
    #Authentication.style.button_color = 'lightgreen'
    # if bad:
    else:
        error.append_stdout("Invalid API token")
        Authentication.description='NOT Authorized'
        Authentication.button_style='warning' # 'success', 'info', 'warning', 'danger' or ''
        Authentication.icon='unlock'
    # output note 
    


def on_button_clicked(b):
    # clear any output ( error output from authorize )
    error.clear_output()
    authorize()
    
    
Submit.on_click(on_button_clicked)


#https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Styling.html#The-Grid-layout

header  = Authentication
main    = API_widget
sidebar = environ
footer  = Submit



form = GridBox(children=[header, main, sidebar,footer],
        layout=Layout(
            width='80%',
            margin="2% 0% 2% 10%",
            grid_template_rows='auto auto auto',
            grid_template_columns='25% 25% 25% 25%',
            #grid_gap='5px 5px',
            grid_template_areas='''
            " . header header ."
            "main main sidebar sidebar"
            " . footer footer . "
            ''')
       )

display(HTML("<h2 style='text-align: center;'>Please enter your API Credentials</h2>"))
display(form)
display(error)



def on_value_change(change):
    Authentication.description='NOT Authorized'
    Authentication.button_style='warning' # 'success', 'info', 'warning', 'danger' or ''
    Authentication.icon='unlock'
    headers = {
    'Authorization': 'Bearer %s' % (change['new']) ,
    }
    

API_widget.observe(on_value_change, names='value')



environs = {'Production':'https://canvas.upenn.edu/api/v1/', 'Test':'b', 'Beta':'c'}
domain = environs[environ.value]


def test_api():
    
    #headers = {
    #'Authorization': 'Bearer %s' % (API_widget.value) ,
    #}
    
    url = domain + 'accounts/self/scopes'
    response = requests.get(url,headers=headers)
    r = response.json()
    #print(response.status_code)
    return response.status_code



######################################
######### HELPER FUNCTION ############
######################################

# utc to est convert and back 
# all user input is in est in the basic time format
# example: 2019-05-14 1am
# output from canvas 2019-05-14T14:28:08Z (utc)

def est_to_utc(datestring):
    utc=pytz.utc
    eastern=pytz.timezone('US/Eastern')
    fmt='%Y-%m-%d %H:%M'
    date=dt.datetime.strptime(datestring,"%Y-%m-%d %I%p")
    date_eastern=eastern.localize(date,is_dst=None)
    date_utc=date_eastern.astimezone(utc)
    return(date_utc.strftime(fmt))
    


GridBox(children=(Button(button_style='warning', description='NOT Authorized', disabled=True, icon='unlock', l…

Output()

<IPython.core.display.Javascript object>

<hr>

## Explore Endpoints 🔎
##### Description
Once you have validated your API credentials you can start making API requests! 

Feel free to explore what can be done with the Canvas API by setting the drop down values below 

**purpose:** list of all of the api endpoints and their associated info


In [44]:
#GET /api/v1/accounts/:account_id/scopes



#print(headers)
#print(domain)
output_df = widgets.Output()

ALL = 'ALL'
def unique_sorted_values_plus_ALL(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, ALL)
    return unique

try:
    headers
except NameError:
    headers = {}
    with output_df:
        display(HTML("<p style='text-align: center;'>No API Token set</p>"))
url = domain +'accounts/self/scopes'
response = requests.get(url,headers=headers)
r = response.json()
df_scopes = pd.DataFrame.from_dict(r, orient='columns')

try:
    df_scopes.resource_name 
except AttributeError:
    df_scopes = pd.DataFrame.from_dict([{'resource_name':'api token error','verb':'api token error'}], orient='columns')


dropdown_resource = widgets.Dropdown(description='Resource',
                                     options = unique_sorted_values_plus_ALL(df_scopes.resource_name),
                                    layout=Layout(width='40%',padding="5px 20px 20px 20px"))
dropdown_apimethod = widgets.Dropdown(description='API Method',
                                     options = unique_sorted_values_plus_ALL(df_scopes.verb),
                                    layout=Layout(width='40%',padding="5px 20px 20px 20px"))




def common_filtering(resource, verb):
    output_df.clear_output(wait=True)
    
    # year -> resource 
    # purpose -> verb
    if (resource == ALL) & (verb == ALL):
        common_filter = df_scopes
    elif (resource == ALL):
        common_filter = df_scopes[df_scopes.verb == verb]
    elif (verb == ALL):
        common_filter = df_scopes[df_scopes.resource_name == resource]
    else:
        common_filter = df_scopes[(df_scopes.resource_name == resource) & 
                                  (df_scopes.verb == verb)]
    #common_filter = df_scopes
    if common_filter.empty:
        with output_df:
            display(HTML("<p style='text-align: center;'>No matching results</p>"))
        
    else:
        with output_df:
            display(common_filter)


def dropdown_resource_eventhandler(change):
    common_filtering(change.new, dropdown_apimethod.value)


def dropdown_apimethod_eventhandler(change):
    common_filtering(dropdown_resource.value, change.new)

#pp(r)

dropdown_resource.observe(dropdown_resource_eventhandler, names='value')
dropdown_apimethod.observe(dropdown_apimethod_eventhandler, names='value')


display(HTML("<h2 style='text-align: center;'>Explore the Endpoints Available in the Canvas API based on your credientials</h2>"))
#if headers =={}:

display(dropdown_resource)

display(dropdown_apimethod)
#load_output()

Dropdown(description='Resource', layout=Layout(padding='5px 20px 20px 20px', width='40%'), options=('ALL', 'AP…

Dropdown(description='API Method', layout=Layout(padding='5px 20px 20px 20px', width='40%'), options=('ALL', '…

In [43]:
from IPython.core.display import HTML

display(HTML("<h3 style='padding-left:40px;'>Output:</h3>"))
display(output_df)

Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': '                              action  …

## Page Views 
#### Description 
[see canvas doc](https://canvas.instructure.com/doc/api/users.html#method.page_views.index)  

currently just shows 25 results

5443278

#### Directions
- put in users canvas id 
- you can leave the fields blank or specify a date ( EST ) ( **CURRENTLY NOT FUNCTIONAL** )
- the out put will be in utc so use the quick utc to est converter 
- Page views are returned in descending order, newest to oldest.
- Pagination is not possible please contact Maddy if needed

###### Why Pagination isn't great in pageviews
technically there is no page 1, page 2, etc. There are bookmarks because your pageviews could be constantly changing


In [41]:
# /api/v1/accounts/:account_id/scopes

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 999)
#df_pageview = pd.DataFrame.from_dict({}, orient='columns')

pageview_output = widgets.Output(layout={'height':'100%'})#layout={'height':'100%'}
userinfo_output = widgets.Output()
pagination_output = widgets.Output()

user_id = widgets.Text(placeholder='user_id',
                       layout=Layout(width='90%',align_items='center', grid_area='header'))

start_date = widgets.DatePicker(
    description='start date',
    disabled=False,)
start_time = widgets.Dropdown(
    options=['none','1am', '2am', '3am', '4am', '5am', '6am', '7am', '8am', '9am', '10am', '11am', '12am','1pm', '2pm', '3pm', '4pm', '5pm', '6pm', '7pm', '8pm', '9pm', '10pm', '11pm', '12pm'],
    value='none',
    description='start time',
    disabled=False,)

end_date = widgets.DatePicker(
    description='end date',
    disabled=False,)
end_time = widgets.Dropdown(
    options=['none','1am', '2am', '3am', '4am', '5am', '6am', '7am', '8am', '9am', '10am', '11am', '12am','1pm', '2pm', '3pm', '4pm', '5pm', '6pm', '7pm', '8pm', '9pm', '10pm', '11pm', '12pm'],
    value='none',
    description='end time',
    disabled=False,)

Submit = Button(
    value=False,
    description='Get PageViews',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    icon='',
    layout=Layout(width='auto', grid_area='footer')
)


header = user_id
left = widgets.VBox([start_date, start_time],layout=Layout( grid_area='left'))#layout=Layout(width='90%',align_items='center', grid_area='left')
right = widgets.VBox([end_date, end_time],layout=Layout( grid_area='right'))
footer = Submit

form = GridBox(children=[header, left, right,footer],
        layout=Layout(
            width='80%',
            margin="2% 0% 2% 10%",
            grid_template_rows='auto auto auto',
            grid_template_columns='25% 25% 25% 25%',
            #grid_gap='5px 5px',
            grid_template_areas='''
            " . header header ."
            "left left right right"
            " . footer footer . "
            ''')
       )

display(HTML("<h2 style='text-align: center;'>Explore the Pageview History</h2>"))

display(form)
display(HTML('<p><b>NOTE</b>: if you leave the date time fields blank it will get the most current results'))
#display(pagination_output)



def est_to_utc(datestring):
    utc=pytz.utc
    eastern=pytz.timezone('US/Eastern')
    fmt='%Y-%m-%d'# %H:%M' <- ADD THE TIME LATER
    date=dt.datetime.strptime(datestring,"%Y-%m-%d %I%p")
    date_eastern=eastern.localize(date,is_dst=None)
    date_utc=date_eastern.astimezone(utc)
    return(date_utc.strftime(fmt))
    

def get_params():
    #basic logic on the start and end time if available 
    params = {}
    if start_date.value:
        start = est_to_utc(str(start_date.value)+' '+'12am')
        params['start_time'] = start
    if end_date.value:
        end = est_to_utc(str(end_date.value)+' '+'12am')
        params['end_time'] = end
    return params
    
    

def page_views(user_id):
    # GET /api/v1/users/:user_id/page_views
    params = get_params()
    url = domain + 'users/'+user_id+'/page_views'
    params['per_page'] = '25'
    print("params",params)
    response = requests.get(url,headers=headers,params=params)
    print(response.url)
    return response
    
    
def user_info(user_id):
    #url:GET|/api/v1/users/:id
    url = domain + 'users/'+user_id
    response = requests.get(url,headers=headers)
    print(url)
    return(response.json())
    #pp(response.json())
    
def on_button_clicked(b):
    # clear any output ( error output from authorize )
    pageview_output.clear_output()
    userinfo_output.clear_output()
    # Validate
    """
    with pagination_output:
        display(
            Button(
                value=False,
                description='ok',
                disabled=False, 
                button_style='info', # 'success', 'info', 'warning', 'danger' or ''
                icon='',))
    """
    
    user = user_id.value
    if not user:
        print("yeah")
        with userinfo_output:
            display(HTML("<h4 ><span style='color:red;'>ERROR:</span> No User Set</h4>"))
    else:
        userinfo = user_info(user)
        #print("user",user)
        response = page_views(user)
        print("STATUS CODE: ",response.status_code)
        #pp(response.links)
        # format 
        r = response.json()
        print("r",r)
        # flatten results
        df_pageview = json_normalize(r)
        print("r",r)
        #print(userinfo)
        with userinfo_output:
            if userinfo:
                display(HTML('<h4> Page Results for ' + userinfo['name'] + ' (' +userinfo['login_id'] +')</h4>'))
            else:
                 display(HTML('<h4> Page Results for <i>NONE</i></h4>'))       
        with pageview_output:
            
            # Removing column
            
            list_drop = ["session_id", "developer_key_id","summarized","id","asset_user_access_id","user_request","asset_type"]
            df_pageview.drop(list_drop, axis=1, inplace=True)
            df_pageview['updated_at'] = pd.to_datetime(df_pageview['updated_at'], format='%Y-%m-%d %H:%M:%S', utc=True)\
                        .dt.tz_convert('America/New_York')\
                        .dt.strftime('%Y-%m-%d %I:%M%p')
            #            #.dt.tz_localize('UTC')\
            #            #.dt.tz_convert('EST')\
            #            #.dt.strftime('%Y-%m-%d %I:%M%p')
            df_pageview['created_at'] = pd.to_datetime(df_pageview['created_at'], format='%Y-%m-%d %H:%M:%S', utc=True)\
                        .dt.tz_convert('America/New_York')\
                        .dt.strftime('%Y-%m-%d %I:%M%p')
            # .tz_convert('UTC').tz_convert(None)
            #df.tz_localize('EST', level=0)
            display(df_pageview)
        #display(df_pageview) 
        
        # clear_output(wait=True) 


Submit.on_click(on_button_clicked)

#5443278
# should have pagination somewhere 



GridBox(children=(Text(value='', layout=Layout(align_items='center', grid_area='header', width='90%'), placeho…

https://canvas.upenn.edu/api/v1/users/5279847
params {'per_page': '25'}
https://canvas.upenn.edu/api/v1/users/5279847/page_views?per_page=25
STATUS CODE:  200
r [{'session_id': '623169950df3030a515856f6441e8056', 'url': 'https://canvas.upenn.edu/courses/1440507/assignments/6771817', 'context_type': 'Course', 'asset_type': None, 'controller': 'assignments', 'action': 'show', 'interaction_seconds': None, 'created_at': '2019-05-16T18:20:19Z', 'updated_at': '2019-05-16T18:20:19Z', 'developer_key_id': None, 'user_request': None, 'render_time': 1.252563, 'user_agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36', 'asset_user_access_id': 267342359, 'participated': False, 'summarized': None, 'http_method': 'get', 'remote_ip': '128.91.185.159', 'id': 'cf4af1db-ac55-4b2b-9edc-5124f444b779', 'contributed': False, 'links': {'user': 5279847, 'context': 1440507, 'asset': None, 'real_user': None, 'account': 96678}, 'app_na

In [42]:
display(HTML("<h3 style='padding-left:40px;'>Output (PageViews):</h3>"))

# class output output_scroll set to 24 -- shouldnt 
#pd.set_option('display.max_rows', 999)
#print(pageview_output.layout)

download = Button(
    value=False,
    description='Download History',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    icon='fa-download',)
filename = widgets.Text(value='pageview_data.csv',description='filename')

accordion = widgets.Accordion(children=[widgets.HBox([filename,download])])
accordion.set_title(0, 'Download')



def on_download_clicked(b):
    #download to csv
    print("yeahhhh")
    df_pageview.to_csv("output_filename.csv", index=False, encoding='utf8')
    #display(filename)

download.on_click(on_download_clicked)
if headers == {}:
    display(HTML("<p style='text-align: center;'>No API Token set</p>"))
else:
    #display(HTML("<style>.rendered_html th {max-width: 120px;}</style>"))
    display(userinfo_output)
    #display(pagination_output)
    display(pageview_output)
    
    display(accordion)
    
    

    
#2019-05-14 10:02AM    
# 2019-05-14T15:01:53Z	

#2019-05-14T16:31:17Z	
#2019-05-14 11:31:17-05:00	

Output()

Output(layout=Layout(height='100%'))

Accordion(children=(HBox(children=(Text(value='pageview_data.csv', description='filename'), Button(button_styl…

In [37]:

url = domain +'accounts/self/scopes'
response = requests.get(url,headers=headers)
r = response.json()
#print(r)
#ok = pd.read_json(response)
#ok
#/api/v1/courses/:course_id/quizzes/:quiz_id/submission
#https://canvas.upenn.edu/courses/1424853/quizzes/2118685
user = '5507597'
course_id = '1424853'
quiz_id ='2118685'

url = '%s/courses/%s/quizzes/%s/submissions' % (domain,course_id,quiz_id)

params= {
    "per_page": 100,
}
#"include[user]": user 

response = requests.get(url,headers=headers,params=params)
print(response.url)


https://canvas.upenn.edu/api/v1//courses/1424853/quizzes/2118685/submissions?per_page=100


In [38]:

display(pageview_output)

Output(layout=Layout(height='100%'))

In [10]:
df_pageview.to_csv('example.csv')

NameError: name 'df_pageview' is not defined

In [ ]:
#response.links
display("ok")

## Submission History
#### Description 


#### Directions

In [ ]:
# create form and make requests




In [ ]:
# display results

## Template
#### Description 


#### Directions

In [ ]:
# Output data to a CSV file
# Typically, I don't want row numbers in my output file, hence index=False.
# To avoid character issues, I typically use utf8 encoding for input/output.
data.to_csv("output_filename.csv", index=False, encoding='utf8')
# Output data to an Excel file.
# For the excel output to work, you may need to install the "xlsxwriter" package.
data.to_csv("output_excel_file.xlsx", sheet_name="Sheet 1", index=False)

###### To Do 
- finish page views
- download link 
- feedback
###### SOURCES
 - https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20Basics.html
 - https://canvas.instructure.com/doc/api/file.oauth.html#using-access-tokens
 - https://towardsdatascience.com/bring-your-jupyter-notebook-to-life-with-interactive-widgets-bc12e03f0916
 
 ###### I like 
 - https://jupyter-dashboards-layout.readthedocs.io/en/latest/getting-started.html
 - https://jupyter-notebook.readthedocs.io/en/stable/extending/frontend_extensions.html

In [ ]:
x = {'a':'b'}
if x == {}:
    print('1')
if not x:
    print('2')


<hr>  

[Canvas at Penn](http://infocanvas.upenn.edu) - Penn's Source for Courseware Innovation & News   
Maddy Hodges' [Github Repo](https://github.com/Mfhodges/canvas_api_jupyter)